<a href="https://colab.research.google.com/github/tm-ty/colab-notebooks-tm/blob/main/%F0%9F%94%A5Crawler_em_Massa_(Crawl4AI)_(Parallel_Crawling_Batch)(working)_v5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# v5 - by Claude
**Instruções de Uso:**

1. Abra um novo notebook no Google Colab.
2. Copie e cole o código fornecido em uma célula do notebook.
3. Execute a célula (Ctrl+Enter ou clique no botão de play).
4. O código irá:
   - Instalar as dependências necessárias
   - Buscar automaticamente as URLs do sitemap da Pydantic AI
   - Rastrear cada página
   - Gerar arquivos Markdown
   - Criar um arquivo ZIP com todos os Markdowns
5. Ao final da execução, um arquivo `crawl4ai_output.zip` será disponibilizado para download.
6. Se solicitado, reinicie a sessão do Colab após a instalação das dependências.

**Personalização:**

Para usar em outro site, altere a URL do sitemap na linha:
```python
sitemap_url = "https://ai.pydantic.dev/sitemap.xml"
```

## v5 - by Claude - CÓDIGO COMPLETO EM UMA ÚNICA CÉLULA

In [ ]:
# Instalar dependências
!pip install -U crawl4ai requests
!crawl4ai-setup
!crawl4ai-doctor
!python -m playwright install --with-deps chromium

# Importar bibliotecas
import asyncio
from typing import List
from crawl4ai import AsyncWebCrawler, BrowserConfig, CrawlerRunConfig
from crawl4ai.markdown_generation_strategy import DefaultMarkdownGenerator
import os
import shutil
from google.colab import files
import requests
from xml.etree import ElementTree

# Definir o diretório temporário para salvar os arquivos
temp_output_directory = "/content/crawl4ai_temp_output"

# Função para obter URLs do sitemap
def get_urls_from_sitemap(sitemap_url: str) -> List[str]:
    try:
        response = requests.get(sitemap_url)
        response.raise_for_status()

        # Analisar o XML
        root = ElementTree.fromstring(response.content)

        # Extrair todas as URLs do sitemap
        namespace = {'ns': 'http://www.sitemaps.org/schemas/sitemap/0.9'}
        urls = [loc.text for loc in root.findall('.//ns:loc', namespace)]

        return urls
    except Exception as e:
        print(f"Erro ao buscar o sitemap: {e}")
        return []

# Função para rastreamento sequencial
async def crawl_sequential(urls: List[str]):
    print("\n=== Rastreamento Sequencial com Reutilização de Sessão ===")

    browser_config = BrowserConfig(
        headless=True,
        extra_args=["--disable-gpu", "--disable-dev-shm-usage", "--no-sandbox"],
    )

    crawl_config = CrawlerRunConfig(
        markdown_generator=DefaultMarkdownGenerator()
    )

    # Criar o crawler (abre o navegador)
    crawler = AsyncWebCrawler(config=browser_config)
    await crawler.start()

    try:
        session_id = "session1"  # Reutilizar a mesma sessão em todas as URLs
        for url in urls:
            result = await crawler.arun(
                url=url,
                config=crawl_config,
                session_id=session_id
            )
            if result.success:
                print(f"Rastreado com sucesso: {url}")
                print(f"Tamanho do Markdown: {len(result.markdown_v2.raw_markdown)}")

                # Salvar o arquivo temporariamente
                filename = os.path.join(temp_output_directory, f"{url.replace('/', '_').replace(':', '_')}.md")
                os.makedirs(os.path.dirname(filename), exist_ok=True)
                with open(filename, "w", encoding="utf-8") as f:
                    f.write(result.markdown_v2.raw_markdown)
                print(f"Markdown salvo em: {filename}")

            else:
                print(f"Falha: {url} - Erro: {result.error_message}")
    finally:
        # Depois que todas as URLs forem concluídas, fechar o crawler (e o navegador)
        await crawler.close()

# Função principal
async def main():
    sitemap_url = "https://ai.pydantic.dev/sitemap.xml" # <<-- INSIRA SUA URL NESSA LINHA
    urls = get_urls_from_sitemap(sitemap_url)

    if urls:
        print(f"Encontradas {len(urls)} URLs para rastrear")
        await crawl_sequential(urls)

        # Compactar a pasta de saída
        shutil.make_archive("crawl4ai_output", 'zip', temp_output_directory)
        print(f"Diretório de saída compactado para crawl4ai_output.zip")

        # Disponibilizar para download
        files.download("crawl4ai_output.zip")
    else:
        print("Nenhuma URL encontrada para rastrear")

# Executar a função principal
await main()

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.9 MB/s eta 0:00:00
  Created wheel for pytest-mockito: filename=pytest_mockito-0.0.4-py3-none-any.whl size=3700 sha256=6f8c7fbf39572386d67f21fa6152ebf1a22927be2f3df9567fb2c677175611a5
  Stored in directory: /root/.cache/pip/wheels/df/72/3d/ad383ec25e3ebb14d12326426166be

[INIT].... → Running post-installation setup...
[INIT].... → Installing Playwright browsers...
Installing dependencies...
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [61.9 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,200 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,599 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## v5 - by Claude - CÓDIGO COMPLETO **COM SEPARAÇÃO** DE CÉLULAS

### 1. Célula de dependências:

In [ ]:
# Instalar dependências
!pip install -U crawl4ai requests
!crawl4ai-setup
!crawl4ai-doctor
!python -m playwright install --with-deps chromium

### 2. Células de código principal:

In [ ]:
# Importar bibliotecas
import asyncio
from typing import List
from crawl4ai import AsyncWebCrawler, BrowserConfig, CrawlerRunConfig
from crawl4ai.markdown_generation_strategy import DefaultMarkdownGenerator
import os
import shutil
from google.colab import files
import requests
from xml.etree import ElementTree

# Definir o diretório temporário para salvar os arquivos
temp_output_directory = "/content/crawl4ai_temp_output"

# ... (o restante do seu código)

### 3. Fluxo de trabalho recomendado:

In [ ]:
# Função para obter URLs do sitemap
def get_urls_from_sitemap(sitemap_url: str) -> List[str]:
    try:
        response = requests.get(sitemap_url)
        response.raise_for_status()

        # Analisar o XML
        root = ElementTree.fromstring(response.content)

        # Extrair todas as URLs do sitemap
        namespace = {'ns': 'http://www.sitemaps.org/schemas/sitemap/0.9'}
        urls = [loc.text for loc in root.findall('.//ns:loc', namespace)]

        return urls
    except Exception as e:
        print(f"Erro ao buscar o sitemap: {e}")
        return []

# Função para rastreamento sequencial
async def crawl_sequential(urls: List[str]):
    print("\n=== Rastreamento Sequencial com Reutilização de Sessão ===")

    browser_config = BrowserConfig(
        headless=True,
        extra_args=["--disable-gpu", "--disable-dev-shm-usage", "--no-sandbox"],
    )

    crawl_config = CrawlerRunConfig(
        markdown_generator=DefaultMarkdownGenerator()
    )

    # Criar o crawler (abre o navegador)
    crawler = AsyncWebCrawler(config=browser_config)
    await crawler.start()

    try:
        session_id = "session1"  # Reutilizar a mesma sessão em todas as URLs
        for url in urls:
            result = await crawler.arun(
                url=url,
                config=crawl_config,
                session_id=session_id
            )
            if result.success:
                print(f"Rastreado com sucesso: {url}")
                print(f"Tamanho do Markdown: {len(result.markdown_v2.raw_markdown)}")

                # Salvar o arquivo temporariamente
                filename = os.path.join(temp_output_directory, f"{url.replace('/', '_').replace(':', '_')}.md")
                os.makedirs(os.path.dirname(filename), exist_ok=True)
                with open(filename, "w", encoding="utf-8") as f:
                    f.write(result.markdown_v2.raw_markdown)
                print(f"Markdown salvo em: {filename}")

            else:
                print(f"Falha: {url} - Erro: {result.error_message}")
    finally:
        # Depois que todas as URLs forem concluídas, fechar o crawler (e o navegador)
        await crawler.close()

# Função principal
async def main():
    sitemap_url = "https://ai.pydantic.dev/sitemap.xml" # <<-- INSIRA SUA URL NESSA LINHA
    urls = get_urls_from_sitemap(sitemap_url)

    if urls:
        print(f"Encontradas {len(urls)} URLs para rastrear")
        await crawl_sequential(urls)

        # Compactar a pasta de saída
        shutil.make_archive("crawl4ai_output", 'zip', temp_output_directory)
        print(f"Diretório de saída compactado para crawl4ai_output.zip")

        # Disponibilizar para download
        files.download("crawl4ai_output.zip")
    else:
        print("Nenhuma URL encontrada para rastrear")

# Executar a função principal
await main()

# v5 by Gemini Exp 1206 - CÓDIGO COMPLETO EM UMA ÚNICA CÉLULA

**Como executar:**

1. Execute a célula de instalação de dependências **uma vez**. (Se solicitado, reinicie a sessão do Colab após a instalação das dependências.)
2. Execute as demais células.
3. O código irá:
    *   Obter as URLs do `sitemap.xml`.
    *   Rastrear cada URL.
    *   Salvar os arquivos Markdown temporariamente em `/content/crawl4ai_temp_output`.
    *   Criar um arquivo `crawl4ai_output.zip` contendo todos os arquivos Markdown.
    *   Iniciar o download do arquivo `crawl4ai_output.zip` no seu navegador.

Agora você tem um processo automatizado que obtém as URLs do sitemap, rastreia cada uma delas e fornece os resultados em um arquivo zip para download!


## v5 by Gemini Exp 1206 - CÓDIGO COMPLETO EM UMA ÚNICA CÉLULA

In [ ]:
# Instalar dependências (execute esta célula apenas uma vez)
!pip install -U crawl4ai requests
!crawl4ai-setup
!crawl4ai-doctor
!python -m playwright install --with-deps chromium

# Importar bibliotecas
import asyncio
from typing import List
from crawl4ai import AsyncWebCrawler, BrowserConfig, CrawlerRunConfig
from crawl4ai.markdown_generation_strategy import DefaultMarkdownGenerator
import os
import shutil
from google.colab import files
import requests
from xml.etree import ElementTree

# Definir o diretório temporário para salvar os arquivos
temp_output_directory = "/content/crawl4ai_temp_output"

# Função para obter URLs do sitemap
def get_urls_from_sitemap(sitemap_url: str) -> List[str]:
    """
    Fetches all URLs from a given sitemap URL.

    Args:
        sitemap_url: URL of the sitemap.

    Returns:
        List[str]: List of URLs found in the sitemap.
    """
    try:
        response = requests.get(sitemap_url)
        response.raise_for_status()  # Raise an exception for bad status codes

        # Parse the XML
        root = ElementTree.fromstring(response.content)

        # Extract all URLs from the sitemap
        # The namespace is usually defined in the root element
        namespace = {'ns': 'http://www.sitemaps.org/schemas/sitemap/0.9'}
        urls = [loc.text for loc in root.findall('.//ns:loc', namespace)]

        return urls
    except Exception as e:
        print(f"Error fetching or parsing sitemap: {e}")
        return []

# Função para rastreamento sequencial
async def crawl_sequential(urls: List[str]):
    print("\n=== Sequential Crawling with Session Reuse ===")

    browser_config = BrowserConfig(
        headless=True,
        extra_args=["--disable-gpu", "--disable-dev-shm-usage", "--no-sandbox"],
    )

    crawl_config = CrawlerRunConfig(
        markdown_generator=DefaultMarkdownGenerator()
        # Sem output_dir, vamos salvar os arquivos temporariamente e depois zipá-los
    )

    # Criar o crawler (abre o navegador)
    crawler = AsyncWebCrawler(config=browser_config)
    await crawler.start()

    try:
        session_id = "session1"  # Reutilizar a mesma sessão em todas as URLs
        for url in urls:
            result = await crawler.arun(
                url=url,
                config=crawl_config,
                session_id=session_id
            )
            if result.success:
                print(f"Successfully crawled: {url}")
                print(f"Markdown length: {len(result.markdown_v2.raw_markdown)}")

                # Salvar o arquivo temporariamente
                filename = os.path.join(temp_output_directory, f"{url.replace('/', '_').replace(':', '_')}.md")
                os.makedirs(os.path.dirname(filename), exist_ok=True)
                with open(filename, "w") as f:
                    f.write(result.markdown_v2.raw_markdown)
                print(f"Markdown saved to: {filename}")

            else:
                print(f"Failed: {url} - Error: {result.error_message}")
    finally:
        # Depois que todas as URLs forem concluídas, fechar o crawler (e o navegador)
        await crawler.close()

# Função principal
async def main():
    sitemap_url = "https://ai.pydantic.dev/sitemap.xml"  # URL do sitemap
    urls = get_urls_from_sitemap(sitemap_url)

    if urls:
        print(f"Found {len(urls)} URLs to crawl")
        await crawl_sequential(urls)

        # Compactar a pasta de saída
        shutil.make_archive("crawl4ai_output", 'zip', temp_output_directory)
        print(f"Output directory zipped to crawl4ai_output.zip")

        # Disponibilizar para download
        files.download("crawl4ai_output.zip")

        # Limpar o diretório temporário (opcional)
        shutil.rmtree(temp_output_directory)
        print(f"Temporary output directory removed: {temp_output_directory}")
    else:
        print("No URLs found to crawl")

# Executar a função principal
await main()

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.1 MB/s eta 0:00:00
  Created wheel for pytest-mockito: filename=pytest_mockito-0.0.4-py3-none-any.whl size=3700 sha256=9c2fa3bd10cbf84431954ad50057eba4af7ef07e541c0277c5227ac73c768356
  Stored in directory: /root/.cache/pip/wheels/df/72/3d/ad383ec25e3ebb14d12326426166be

[INIT].... → Running post-installation setup...
[INIT].... → Installing Playwright browsers...
Installing dependencies...
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:6 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:7 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/multiverse amd64 Packages [45.2 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,560 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Temporary output directory removed: /content/crawl4ai_temp_output


## v5 by Gemini Exp 1206 - CÓDIGO COMPLETO **COM SEPARAÇÃO** DE CÉLULAS

### 1. Célula de dependências:

In [ ]:
# Instalar dependências (execute esta célula apenas uma vez)
!pip install -U crawl4ai requests
!crawl4ai-setup
!crawl4ai-doctor
!python -m playwright install --with-deps chromium

### 2. Células de código principal:

In [ ]:
# Importar bibliotecas
import asyncio
from typing import List
from crawl4ai import AsyncWebCrawler, BrowserConfig, CrawlerRunConfig
from crawl4ai.markdown_generation_strategy import DefaultMarkdownGenerator
import os
import shutil
from google.colab import files
import requests
from xml.etree import ElementTree

# Definir o diretório temporário para salvar os arquivos
temp_output_directory = "/content/crawl4ai_temp_output"

# ... (o restante do seu código)

### 3. Fluxo de trabalho recomendado:

In [ ]:
# Importar bibliotecas
import asyncio
from typing import List
from crawl4ai import AsyncWebCrawler, BrowserConfig, CrawlerRunConfig
from crawl4ai.markdown_generation_strategy import DefaultMarkdownGenerator
import os
import shutil
from google.colab import files
import requests
from xml.etree import ElementTree

# Definir o diretório temporário para salvar os arquivos
temp_output_directory = "/content/crawl4ai_temp_output"

# Função para obter URLs do sitemap
def get_urls_from_sitemap(sitemap_url: str) -> List[str]:
    """
    Fetches all URLs from a given sitemap URL.

    Args:
        sitemap_url: URL of the sitemap.

    Returns:
        List[str]: List of URLs found in the sitemap.
    """
    try:
        response = requests.get(sitemap_url)
        response.raise_for_status()  # Raise an exception for bad status codes

        # Parse the XML
        root = ElementTree.fromstring(response.content)

        # Extract all URLs from the sitemap
        # The namespace is usually defined in the root element
        namespace = {'ns': 'http://www.sitemaps.org/schemas/sitemap/0.9'}
        urls = [loc.text for loc in root.findall('.//ns:loc', namespace)]

        return urls
    except Exception as e:
        print(f"Error fetching or parsing sitemap: {e}")
        return []

# Função para rastreamento sequencial
async def crawl_sequential(urls: List[str]):
    print("\n=== Sequential Crawling with Session Reuse ===")

    browser_config = BrowserConfig(
        headless=True,
        extra_args=["--disable-gpu", "--disable-dev-shm-usage", "--no-sandbox"],
    )

    crawl_config = CrawlerRunConfig(
        markdown_generator=DefaultMarkdownGenerator()
        # Sem output_dir, vamos salvar os arquivos temporariamente e depois zipá-los
    )

    # Criar o crawler (abre o navegador)
    crawler = AsyncWebCrawler(config=browser_config)
    await crawler.start()

    try:
        session_id = "session1"  # Reutilizar a mesma sessão em todas as URLs
        for url in urls:
            result = await crawler.arun(
                url=url,
                config=crawl_config,
                session_id=session_id
            )
            if result.success:
                print(f"Successfully crawled: {url}")
                print(f"Markdown length: {len(result.markdown_v2.raw_markdown)}")

                # Salvar o arquivo temporariamente
                filename = os.path.join(temp_output_directory, f"{url.replace('/', '_').replace(':', '_')}.md")
                os.makedirs(os.path.dirname(filename), exist_ok=True)
                with open(filename, "w") as f:
                    f.write(result.markdown_v2.raw_markdown)
                print(f"Markdown saved to: {filename}")

            else:
                print(f"Failed: {url} - Error: {result.error_message}")
    finally:
        # Depois que todas as URLs forem concluídas, fechar o crawler (e o navegador)
        await crawler.close()

# Função principal
async def main():
    sitemap_url = "https://ai.pydantic.dev/sitemap.xml"  # URL do sitemap
    urls = get_urls_from_sitemap(sitemap_url)

    if urls:
        print(f"Found {len(urls)} URLs to crawl")
        await crawl_sequential(urls)

        # Compactar a pasta de saída
        shutil.make_archive("crawl4ai_output", 'zip', temp_output_directory)
        print(f"Output directory zipped to crawl4ai_output.zip")

        # Disponibilizar para download
        files.download("crawl4ai_output.zip")

        # Limpar o diretório temporário (opcional)
        shutil.rmtree(temp_output_directory)
        print(f"Temporary output directory removed: {temp_output_directory}")
    else:
        print("No URLs found to crawl")

# Executar a função principal
await main()